In [1]:
import torch
import os, sys
import numpy as np

In [2]:
HOME_DIRECTORY=os.path.abspath(os.path.join(os.getcwd(), os.pardir))

os.chdir(HOME_DIRECTORY)

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # sync ids with nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["MKL_SERVICE_FORCE_INTEL"]="1"

In [4]:
# script params
port=5015
sampling_fn="uncertainty"
lSet_partition=1
base_seed=1
num_GPU=1
al_iterations=4
num_aml_trials=5 #50
budget_size=2500

dataset="CIFAR10"
init_partition=10
step_partition=10
clf_epochs=5 #150
num_classes=10
swa_lr=5e-4
swa_freq=50
swa_epochs=5 #50

log_iter=40

#Data arguments
train_dir=f"{HOME_DIRECTORY}/data/{dataset}/train-{dataset}/"
test_dir=f"{HOME_DIRECTORY}/data/{dataset}/test-{dataset}/"
lSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/lSet_{dataset}.npy"
uSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/uSet_{dataset}.npy"
valSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/valSet_{dataset}.npy"

out_dir=f"{HOME_DIRECTORY}/sample_budgetsize_results"

model_style="vgg_style"
model_type="vgg"
model_depth=16


In [7]:
# It is important to note that we should point results for budget size experiment.
# For example: If we don't take care of savepath & assume it points to 10% budget size experiment 
# then running AL for 15% will have no issues but once we go to 20% - we have earlier results which 
# should not be used. So for such experiments we just copy the base (trained on initial labeled data) 
# classifier to new directory and then run any experiments.

!mkdir -p $out_dir/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16
print("Made best_automl_results directory")
print("Copying base classifier started....")
!scp -r sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla $out_dir/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/
print("Copying base classifier finished!")

# DO the copy again but this time for automl_results
!mkdir -p $out_dir/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/trial-0
print("Made auto_ml_results directory")
print("Copying base classifier checkpoints and config started....")
!scp -r sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/config.yaml $out_dir/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/trial-0/
!scp -r sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints $out_dir/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/trial-0/
print("Copying finished!")

print("""
Please remember to change paths in config file.
For example do replace each "sample_results_aml" occurences in paths to "sample_budgetsize_results" # old directory name to new directory name

""")

# Please remember to change paths in config file
# For example do replace each "sample_results_aml" occurences in paths to "sample_budgetsize_results" # old directory name to new directory name

# Please also modify budget size to 2500 in config.py


Made best_automl_results directory
Copying base classifier started....
Copying base classifier finished!
Made auto_ml_results directory
Copying base classifier checkpoints and config started....
Copying finished!

Please remember to change paths in config file.
For example do replace each "sample_results_aml" occurences in paths to "sample_budgetsize_results" # old directory name to new directory name




In [9]:
!python3 $HOME_DIRECTORY/tools/main_aml.py --n_GPU $num_GPU \
--port $port --sampling_fn $sampling_fn --lSet_partition $lSet_partition \
--seed_id $base_seed \
--init_partition $init_partition --step_partition $step_partition \
--dataset $dataset --budget_size $budget_size \
--out_dir $out_dir \
--num_aml_trials $num_aml_trials --num_classes $num_classes \
--al_max_iter $al_iterations \
--model_type $model_type --model_depth $model_depth \
--clf_epochs $clf_epochs \
--eval_period 1 --checkpoint_period 1 \
--lSetPath $lSetPath --uSetPath $uSetPath --valSetPath $valSetPath \
--train_dir $train_dir --test_dir $test_dir \
--dropout_iterations 25 \
--cfg configs/$dataset/$model_style/$model_type/R-18_4gpu_unreg.yaml \
--vaal_z_dim 32 --vaal_vae_bs 64 --vaal_epochs 2 \
--vaal_vae_lr 5e-4 --vaal_disc_lr 5e-4 --vaal_beta 1.0 --vaal_adv_param 1.0 

========= [NO ADVANCED REGULARIZATION TRICK ACTIVATED] =========
~~~ out_dir:  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla
check_aml_path: 
/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints
Auto ml already exists; So skip doing automl for this!
Number of trials found at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla: 1
best_model_path chosen: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/trial-0/checkpoints/vlBest_acc_47.3_model_epoch_0006.pyth
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/users/ext_p

Files already downloaded and verified
==== Loading valDataset ====
Dataset is augmented
--------------------------------------
Preprocess Operations Selected ==>  [ToTensor()]
--------------------------------------
Files already downloaded and verified
==== Loading noAugDataset ====
Dataset is augmented
--------------------------------------
Preprocess Operations Selected ==>  [ToTensor()]
--------------------------------------
Files already downloaded and verified
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/auto_ml_results/lSet_1/start_1/CIFAR10/20.0/uncertainty/vgg_depth_16/vanilla/trial-0/, temp_cfg.CFG_DEST: config.yaml
########### cfg model type: vgg
[model_builder.py: 167]: Model loaded successfully!!
========OPTIMIZER========
optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 2.404835143682204e-05
    weight_decay: 8.99

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "100/118", "loss": 1.526269853115, "lr": 0.000024048351, "top1_err": 56.250000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "110/118", "loss": 1.576009333134, "lr": 0.000024048351, "top1_err": 54.687500000000}
[logging.py:  75]: json_stats: {"_type": "train_epoch", "epoch": "3/5", "loss": 1.553727699216, "lr": 0.000024048351, "top1_err": 57.573333329264}
[logging.py:  75]: json_stats: {"_type": "Val_iter", "epoch": "3/5", "iter": "10/25", "top1_err": 54.750000000000}
[logging.py:  75]: json_stats: {"_type": "Val_iter", "epoch": "3/5", "iter": "20/25", "top1_err": 57.500003814697}
[logging.py:  75]: json_stats: {"_type": "Val_epoch", "epoch": "3/5", "min_top1_err": 56.540000762939, "top1_err": 56.540000762939}
[train_al.py: 529]: Successfully logged numpy arrays!!
~~~ isPruning Flag:  True
~~~ isEvalEpoch:  True
[logging.py:  75]: json_stats: {"_type": "train_iter", "e

Files already downloaded and verified
==== Loading valDataset ====
Dataset is augmented
--------------------------------------
Preprocess Operations Selected ==>  [ToTensor()]
--------------------------------------
Files already downloaded and verified
==== Loading noAugDataset ====
Dataset is augmented
--------------------------------------
Preprocess Operations Selected ==>  [ToTensor()]
--------------------------------------
Files already downloaded and verified
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/auto_ml_results/lSet_1/start_1/CIFAR10/20.0/uncertainty/vgg_depth_16/vanilla/trial-1/, temp_cfg.CFG_DEST: config.yaml
########### cfg model type: vgg
[model_builder.py: 167]: Model loaded successfully!!
========OPTIMIZER========
optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.00011534911629216944
    weight_decay: 5.7

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "390/938", "loss": 2.234471678734, "lr": 0.000115349116, "top1_err": 87.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "400/938", "loss": 2.095782518387, "lr": 0.000115349116, "top1_err": 81.250000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "410/938", "loss": 1.952678203583, "lr": 0.000115349116, "top1_err": 75.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "420/938", "loss": 2.121794223785, "lr": 0.000115349116, "top1_err": 87.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "430/938", "loss": 2.177657723427, "lr": 0.000115349116, "top1_err": 87.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "440/938", "loss": 2.047818183899, "lr": 0.000115349116, "top1_err": 87.500000000000}
[logging.p

...

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "10/98", "loss": 1.665372788906, "lr": 0.000013169428, "top1_err": 59.765625000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "20/98", "loss": 1.672192692757, "lr": 0.000013169428, "top1_err": 62.109375000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "30/98", "loss": 1.676498770714, "lr": 0.000013169428, "top1_err": 60.546875000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "40/98", "loss": 1.627230703831, "lr": 0.000013169428, "top1_err": 59.765625000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "50/98", "loss": 1.608432948589, "lr": 0.000013169428, "top1_err": 57.421875000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "60/98", "loss": 1.668015241623, "lr": 0.000013169428, "top1_err": 62.500000000000}
[logging.py:  75]: jso

Files already downloaded and verified
Number of testing datapoints: 10000
Len(train_loader): 98
[train_al.py: 450]: Start epoch: 1
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "10/98", "loss": 2.304312705994, "lr": 0.000679865026, "top1_err": 87.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "20/98", "loss": 2.282213926315, "lr": 0.000679865026, "top1_err": 86.718750000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "30/98", "loss": 2.274951219559, "lr": 0.000679865026, "top1_err": 87.109375000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "40/98", "loss": 2.275123596191, "lr": 0.000679865026, "top1_err": 85.937500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "50/98", "loss": 2.267823934555, "lr": 0.000679865026, "top1_err": 86.718750000000}
[logging.py:  75]: json_stats: {"_type": "train_iter",

[logging.py:  75]: json_stats: {"_type": "Val_epoch", "epoch": "4/5", "min_top1_err": 66.860000915527, "top1_err": 66.860000915527}
[train_al.py: 529]: Successfully logged numpy arrays!!
~~~ isPruning Flag:  True
~~~ isEvalEpoch:  True
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "10/98", "loss": 1.811712622643, "lr": 0.000679865026, "top1_err": 69.140625000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "20/98", "loss": 1.776156604290, "lr": 0.000679865026, "top1_err": 69.531250000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "30/98", "loss": 1.827074170113, "lr": 0.000679865026, "top1_err": 70.312500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "40/98", "loss": 1.776335000992, "lr": 0.000679865026, "top1_err": 68.750000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "50/98", "loss": 1.786221027374, 

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "20/25", "loss": 2.272160053253, "lr": 0.000404629884, "top1_err": 87.109375000000}
[logging.py:  75]: json_stats: {"_type": "train_epoch", "epoch": "2/5", "loss": 2.274257799606, "lr": 0.000404629884, "top1_err": 86.743999938965}
[logging.py:  75]: json_stats: {"_type": "Val_iter", "epoch": "2/5", "iter": "10/25", "top1_err": 91.500000000000}
[logging.py:  75]: json_stats: {"_type": "Val_iter", "epoch": "2/5", "iter": "20/25", "top1_err": 88.250000000000}
[logging.py:  75]: json_stats: {"_type": "Val_epoch", "epoch": "2/5", "min_top1_err": 88.980000000000, "top1_err": 88.980000000000}
[train_al.py: 529]: Successfully logged numpy arrays!!
~~~ isPruning Flag:  True
~~~ isEvalEpoch:  True
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "10/25", "loss": 2.266567111015, "lr": 0.000404629884, "top1_err": 86.914062500000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch

uSet Activations: 100%|████████████████████| 4063/4063 [00:18<00:00, 225.61it/s]
u_ranks.shape: (32500,)
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/best_automl_results/lSet_1/start_1/CIFAR10/50.0/uncertainty/vgg_depth_16/vanilla/, temp_cfg.CFG_DEST: config.yaml
========BEFORE==========
len(uSEt):  30000
len(lSEt):  12500
After including activeSet -- len(lSet): 15000 and len(uSet): 30000
saving pickle values...
Saving lSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/best_automl_results/lSet_1/start_1/CIFAR10/50.0/uncertainty/vgg_depth_16/vanilla/lSet.npy in numpy format!!
Saving uSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_budgetsize_results/best_automl_results/lSet_1/start_1/CIFAR10/50.0/uncertainty/vgg_depth_16/vanilla/uSet.npy in numpy format!!
Saving activeSet at /nfs/users/ext